# Session 8 - Language modelling with RNNs (Text Generation)

In [1]:
# data processing tools
import string, os 
import pandas as pd
import numpy as np
np.random.seed(42)

# keras module for building LSTM 
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.utils as ku 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# surpress warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

2023-03-24 12:49:21.780595: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Some helper functions

In [2]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower() # return vocabulary in the text if not part of string.punctuation and make lowercase 
    txt = txt.encode("utf8").decode("ascii",'ignore') # make sure utf8 encoding 
    return txt 

def get_sequence_of_tokens(tokenizer, corpus):
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences

def generate_padded_sequences(input_sequences):
    # get the length of the longest sequence
    max_sequence_len = max([len(x) for x in input_sequences])
    # make every sequence the length of the longest on
    input_sequences = np.array(pad_sequences(input_sequences, 
                                            maxlen=max_sequence_len, 
                                            padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, 
                            num_classes=total_words)
    return predictors, label, max_sequence_len

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 
                        10, 
                        input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1)) # when determining weights during training, drop 10% of the weights - makes the model learn better, makes it a bit more difficult so it doesn't overfit as easily 
   
    # Add Output Layer
    model.add(Dense(total_words, 
                    activation='softmax'))

    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam')
    
    return model

def generate_text(seed_text, next_words, model, max_sequence_len): # seed_text like a prompt we give it 
    for _ in range(next_words): # for however many in the range of next words, like 5 
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], 
                                    maxlen=max_sequence_len-1, 
                                    padding='pre')
        predicted = np.argmax(model.predict(token_list),
                                            axis=1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

## Load the data

In [15]:
data_dir = os.path.join("..", "..", "..", "431868", "news_data")

We're then going to load the data one at a time and append *only* the headlines to our list of data.

In [16]:
all_headlines = []
for filename in os.listdir(data_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(data_dir + "/" + filename)
        all_headlines.extend(list(article_df["headline"].values))

We then clean up a little bit and see how many data points we have.

If called unknown, removed it. 

In [17]:
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

8603

This is actually a small amount of data which is why it doesn't work as well. 

We call out ```clean_text()``` function and then inspect the first 10 texts.

In [18]:
corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['my beijing the sacred city',
 '6 million riders a day 1930s technology',
 'seeking a crossborder conference',
 'questions for despite the yuck factor leeches are big in russian medicine',
 'who is a criminal',
 'an antidote to europes populism',
 'the cost of a speech',
 'degradation of the language',
 'on the power of being awful',
 'trump garbles pitch on a revised health bill']

In the processing steps - stripping punctuation makes grammatically incorrect and can be problematic later 

## Tokenize

We're then going to tokenize our data, using the ```Tokenizer()``` class from ```TensorFlow```, about which you can read more [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer).

We then use the ```get_sequence_of_tokens()``` function we defined above, which turns every text into a sequence of tokens based on the vocabulary from the tokenizer.

In [19]:
tokenizer = Tokenizer()
## tokenization
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1 # '+1' needs a way to index words it doesn't know; creates an arbitrary token, called an out of vocabulary token (oov)
# looks like '<unk>' so every token it doesn't recognize, labels it this way so it can deal with unknown words  

In [20]:
inp_sequences = get_sequence_of_tokens(tokenizer, corpus)
inp_sequences[:10]
len(inp_sequences) # have more now tha nthe 8000 whatever headlines, it returns lots of the same documents with varying n_gram lengths
# can see first and second word, then first through third words and so on 

51770

In [21]:
tokenizer.word_index

{'the': 1,
 'a': 2,
 'to': 3,
 'of': 4,
 'in': 5,
 'for': 6,
 'and': 7,
 'on': 8,
 'is': 9,
 'trump': 10,
 'with': 11,
 'new': 12,
 'at': 13,
 'how': 14,
 'what': 15,
 'you': 16,
 'an': 17,
 'from': 18,
 'as': 19,
 'it': 20,
 'trumps': 21,
 'its': 22,
 'your': 23,
 'are': 24,
 'be': 25,
 'not': 26,
 'us': 27,
 'season': 28,
 'that': 29,
 'by': 30,
 'about': 31,
 'but': 32,
 'can': 33,
 'episode': 34,
 'do': 35,
 'this': 36,
 'when': 37,
 'york': 38,
 'up': 39,
 'over': 40,
 'why': 41,
 'no': 42,
 'i': 43,
 'out': 44,
 'more': 45,
 'my': 46,
 'after': 47,
 'will': 48,
 'may': 49,
 'we': 50,
 'or': 51,
 'war': 52,
 'who': 53,
 'his': 54,
 'health': 55,
 'teaching': 56,
 'questions': 57,
 'now': 58,
 'president': 59,
 'was': 60,
 'one': 61,
 'house': 62,
 'get': 63,
 'today': 64,
 'have': 65,
 'should': 66,
 'into': 67,
 'home': 68,
 'all': 69,
 'dont': 70,
 'life': 71,
 'our': 72,
 'has': 73,
 'plan': 74,
 'good': 75,
 'first': 76,
 'gop': 77,
 '1': 78,
 'says': 79,
 'like': 80,
 'white'

We then want to *pad* our input sequences to make them all the same length. Adding 0's to make sure all the input strings are same length as the longest document. If the longest headline is 10 words, headlines of three words will be three numbers with seven 0s. 

In [22]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [23]:
max_sequence_len
predictors # input vectors
label # looks like one hot encoding vectors 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Create model

We then use the ```create_model()``` function created above to initialize a model, telling the model the length of sequences and the total size of the vocabulary.

In [24]:
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 10)            112650    
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 11265)             1137765   
                                                                 
Total params: 1,294,815
Trainable params: 1,294,815
Non-trainable params: 0
_________________________________________________________________


2023-03-24 12:51:42.563648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-24 12:51:42.565019: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-24 12:51:42.566467: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model training is exactly the same as last week, but instead of document labels, we're fitting the model to predict next word.

*NB!* This will take some time to train! It took me 35 minutes on UCloud 32xCPU.

This model.fit function, runs based on previous as well. So if run the following chunk twice, it will run for 200 epochs. 

In [17]:
history = model.fit(predictors, # data like X_train and y_train
                    label, # labels
                    epochs=100, # arbitarily picked this number, with more epochs and time could get more accurate 
                    batch_size=128, # picked this size due to how long it takes 
                    verbose=1)

Epoch 1/100
405/405 [==============================] - 24s 54ms/step - loss: 7.8858
Epoch 2/100
405/405 [==============================] - 21s 53ms/step - loss: 7.4808
Epoch 3/100
405/405 [==============================] - 21s 51ms/step - loss: 7.3973
Epoch 4/100
405/405 [==============================] - 21s 52ms/step - loss: 7.3198
Epoch 5/100
405/405 [==============================] - 21s 52ms/step - loss: 7.2757
Epoch 6/100
405/405 [==============================] - 21s 52ms/step - loss: 7.1382
Epoch 7/100
405/405 [==============================] - 21s 52ms/step - loss: 7.0256
Epoch 8/100
405/405 [==============================] - 21s 52ms/step - loss: 6.9065
Epoch 9/100
405/405 [==============================] - 21s 52ms/step - loss: 6.7823
Epoch 10/100
405/405 [==============================] - 21s 51ms/step - loss: 6.6585
Epoch 11/100
405/405 [==============================] - 21s 52ms/step - loss: 6.5362
Epoch 12/100
405/405 [==============================] - 21s 52ms/step - lo

When the model has trained, we can then use this to generate *new text*.

In [19]:
print (generate_text("danish", 5, model, max_sequence_len)) # try word russia ; here 5 asks for 5 words 

1/1 [==============================] - 0s 28ms/step
Danish Inventor Is Found Guilty In


What does this mean and what can we use this for? 

## Using pre-trained word embeddings

Instead of having the embedding layer as a trainable parameter, we can instead using a *pretrained word embedding* model like ```word2vec```.

In the following examples, we're using [GloVe embeddings](https://nlp.stanford.edu/projects/glove/). These are trained a little differently from ```word2vec``` but they behave in the same way.

In [42]:
path_to_glove_file = os.path.join("..", "..", "..", "431868", "glove_models") 

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

IsADirectoryError: [Errno 21] Is a directory: '../../../431868/glove_models'

We can define some variables that we're going to use later.

With hits and misses, we're counting how many words in the corpus vocabulary have a corresponding GloVe embedding; misses are the words which appear in our vocabulary but which do not have a GloVe embedding.

In [ ]:
num_tokens = total_words
embedding_dim = 100
hits = 0
misses = 0

In [ ]:
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer - notice that this is different
    model.add(Embedding(
            total_words,
            embedding_dim,
            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
            trainable=False,
            input_length=input_len)
    )
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(500))
    model.add(Dropout(0.2))
    
    # Add Output Layer
    model.add(Dense(total_words, 
                    activation='softmax'))

    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam')
    
    return model

In [ ]:
model = create_model(max_sequence_len, total_words)
model.summary()

In [ ]:
history = model.fit(predictors, 
                    label, 
                    epochs=100,
                    batch_size=128, 
                    verbose=1)

In [ ]:
print (generate_text("china", 30, model, max_sequence_len))

For the assignment - one script to train the model and one to load the saved model and generate new text 